In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu

class Net(nn.Module):
    def __init__(self, kernel_size=5):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, 6, kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        linear_input_size = 16 * conv_width * conv_height
        self.fc1 = nn.Linear(linear_input_size, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,
                                 epochs=epochs, optimizer=optimizer, print_results=False)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, optimizer=None, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net = net_generator()
        results = train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size=32)
        all_results.append(results)
        net_training.print_final_results(results)
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

results = cross_validate_net(Net, dataset, repeat=3, epochs=100)
print_validation_results(results)

Training network 1 ...
Final loss: 1.1769, final accuracy: 73.63 %
	ALBATROSS: 51.72%
	BALD EAGLE: 73.33%
	BARN OWL: 81.48%
	EURASIAN MAGPIE: 80.00%
	FLAMINGO: 73.91%
	MALLARD DUCK: 87.50%
	OSTRICH: 69.23%
	PEACOCK: 89.47%
	PELICAN: 35.00%
	TRUMPTER SWAN: 80.00%
Training network 2 ...
Final loss: 1.5047, final accuracy: 70.55 %
	ALBATROSS: 67.86%
	BALD EAGLE: 85.29%
	BARN OWL: 62.50%
	EURASIAN MAGPIE: 82.86%
	FLAMINGO: 77.27%
	MALLARD DUCK: 64.29%
	OSTRICH: 73.33%
	PEACOCK: 87.10%
	PELICAN: 31.82%
	TRUMPTER SWAN: 60.53%
Training network 3 ...
Final loss: 1.5142, final accuracy: 67.12 %
	ALBATROSS: 50.00%
	BALD EAGLE: 64.29%
	BARN OWL: 73.68%
	EURASIAN MAGPIE: 68.42%
	FLAMINGO: 82.86%
	MALLARD DUCK: 65.38%
	OSTRICH: 80.00%
	PEACOCK: 77.50%
	PELICAN: 40.91%
	TRUMPTER SWAN: 57.69%
Losses:  [1.1768746571998074, 1.5046706656887108, 1.5141913515247711]
Loss:  mean: 1.3986, std: 0.1921
Accuracies:  [0.7363013698630136, 0.7054794520547946, 0.6712328767123288]
Accuracy:  mean: 0.7043, std: 0.03

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/test1.csv")